In [248]:
import json
import math
from urllib.request import urlopen
import pandas as pd
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pyvis.network import Network
from colour import Color

In [134]:
def edge_gen(name, reqs):
    return edge_gen_recursive(name, reqs[1:])
    
def edge_gen_recursive(name, reqs):
    result = []
    
    for r in reqs:
        if type(r) is dict:
            result.append([r['id'], name])
            continue;
        
        edge_gen_recursive(name, r[1:])
    
    return result

In [294]:
url = "https://gt-scheduler.github.io/crawler/202302.json"
response = urlopen(url)
data_json = json.loads(response.read())
data = data_json["courses"]

tree = {}
orig_tree = {}
edges = []

for course_name in data.keys():
    reqs = data[course_name][2]
    new_edges = edge_gen(course_name, reqs)
    if len(new_edges) == 0:
        continue
    
    for ne in new_edges:
        if ne[0] not in tree:
            tree[ne[0]] = []
        
        tree[ne[0]].append(ne[1])
        
        if ne[1] not in orig_tree:
            orig_tree[ne[1]] = []
        
        orig_tree[ne[1]].append(ne[0])
    
    edges += new_edges
        
nodes = list(data.keys())

False


In [136]:
def search(c):
    up = tree[c] if c in tree else None
    down = orig_tree[c] if c in orig_tree else None
    
    return up, down

def major_nodes(major=None, nodes=nodes):
    if major == None:
        return nodes
    
    return [k for k in data.keys() if k.split()[0] == major]

def major_prereqs(major=None, edges=edges):
    if major == None:
        return edges
    
    return [e for e in edges if e[1].split()[0] == major]

def ugrad_nodes(nodes=nodes):
    return [k for k in data.keys() if int(k.split()[1][0]) <= 4]

def ugrad_prereqs(edges=edges):
    return [e for e in edges if int(e[1].split()[1][0]) <= 4 or int(e[0].split()[1][0]) <= 4]

def select_classes(classes, d=None):
    d = d if d != None else -1
    
    all_prereq_to = set(classes)
    new_classes = classes
    ndist = {c:0 for c in classes}
    
    i = 0
    while d == -1 or i < d:
        temp_classes = []
        
        while len(new_classes) != 0:
            c = new_classes.pop(0)
            
            if c not in tree:
                continue
                
            for nc in tree[c]:
                ndist[nc] = i + 1
                temp_classes.append(nc)
        
        if len(temp_classes) == 0:
            break
        
        new_classes += temp_classes
        for c in temp_classes:
            all_prereq_to.add(c)
            
        i += 1
    
    s_e = [e for e in edges if e[0] in all_prereq_to]
    
    for s in s_e:
        ndist[s[1]] = ndist[s[0]] + 1

    
    return list(ndist.keys()), s_e, ndist

In [137]:
def plotly_graph(nodes, edges, title, pl_ticks, pl_bar_max, pl_color, pl_size):
    G = nx.DiGraph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    
#     pos = nx.spring_layout(G,k=2, iterations=100)
    pos = nx.nx_agraph.graphviz_layout(G)
#     pos = nx.circular_layout(G)
#     pos = nx.kamada_kawai_layout(G)
#     pos = nx.spiral_layout(G)
#     pos = nx.shell_layout(G)
#     pos = nx.multipartite_layout(G)
#     pos = nx.kamada_kawai_layout(G)

    for n, p in pos.items():
        G.nodes[n]['pos'] = p
    edge_trace = go.Scatter(
        x=[],
        y=[],
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')
    
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])
    
    factor = 0.9  # Normalized location where continuous colorscale should end

    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            cmin=0,
            cmax=(pl_bar_max if pl_bar_max != None else 20)/factor,
            colorscale = [[0, 'rgb(166,206,227, 0.5)'],
                          [0.05, 'rgb(31,120,180,0.5)'],
                          [0.2, 'rgb(178,223,138,0.5)'],
                          [0.5, 'rgb(51,160,44,0.5)'],
                          [factor, 'rgb(251,154,153,0.5)'],
                          [factor, 'rgb(227,26,28,0.5)'],
                          [1, 'rgb(227,26,28,0.5)']
                         ],
            colorbar=dict(
                tickvals = pl_ticks,
                ticks='outside'
            ),
            color=[] if pl_color == None else [pl_color[n] for n in nodes],
            size= 6 if pl_size == None else [10 for n in nodes], #37,
            line=dict(width=0)))
    
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])

    for node, adjacencies in enumerate(G.adjacency()):
        node_trace['marker']['color'] += tuple([len(adjacencies[1])])
        node_info = adjacencies[0]
        node_trace['text'] += tuple([node_info])
        
    fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                title=title,
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=21, l=5, r=5, t=40),
                xaxis=dict(showgrid=False, zeroline=False,
                           showticklabels=False, mirror=True),
                yaxis=dict(showgrid=False, zeroline=False,
    showticklabels=False, mirror=True)))
    fig.show()

In [338]:
def pyvis_graph(nodes, 
                edges, 
                title="output", 
                show_buttons=True, 
                only_physics_buttons=False, 
                pv_bar_max=20,
                pv_color=None,
                pv_size_min=None,
                pv_size_max=20,):
    
    nt = Network(select_menu=True, filter_menu=True)#, notebook=True, cdn_resources="remote")
    pv_color = pv_color if pv_color != None else list(Color("#a6cee3").range_to(Color("#e31a1c"), 1000))
    
    G = nx.DiGraph()
    
#     print("MATH 1501" in nodes)
    colors = {}
    for n in nodes:
        if n not in tree:
            colors[n] = pv_color[0].hex
            continue
           
        if len(tree[n]) >= pv_bar_max:
            colors[n] = pv_color[-1].hex
            continue
            
        colors[n] = pv_color[math.floor(len(tree[n]) / pv_bar_max * len(pv_color))].hex
    
#     print(colors["MATH 1501"])
    
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    size_dict = dict(G.degree)
    
    # for each node and its attributes in the networkx graph
    for node,node_attrs in G.nodes(data=True):
        nt.add_node(str(node),
                    color=colors[node] if node in colors else pv_color[0].hex,
                    size= (((len(tree[node]) + 1 if node in tree else 1)*40)**(1/2)), #math.floor(((size_dict[node]+1)*40)**(1/2)),
                    **node_attrs)
        
    # for each edge and its attributes in the networkx graph
    for source,target,edge_attrs in G.edges(data=True):
        # if value/width not specified directly, and weight is specified, set 'value' to 'weight'
        if not 'value' in edge_attrs and not 'width' in edge_attrs and 'weight' in edge_attrs:
            # place at key 'value' the weight of the edge
            edge_attrs['value']=edge_attrs['weight']
        # add the edge
        nt.add_edge(str(source),str(target),**edge_attrs)
        
    # turn buttons on
    if show_buttons:
        if only_physics_buttons:
            nt.show_buttons(filter_=['physics'])
        else:
            nt.show_buttons()
#     nt.show(title + ".html")
    nt.show(title + ".html")

In [339]:
def display_graph(nodes=nodes, 
                  edges=edges, 
                  title="GT Course Tree", 
                  pv_bar_max=20,
                  pv_color=None,
                  pv_size_min=None,
                  pv_size_max=20,
                  pl_ticks=[0, 1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], 
                  pl_bar_max=20, 
                  pl_color=None, 
                  pl_size=None):
    
#     sizelist = [(len(tree[n]) if n in tree else 0) + (len(orig_tree[n]) if n in orig_tree else 0) + 10 for n in nodes]
    sizelist = [10 for n in nodes]
        
#     plotly_graph(G, nodes, edges, title, pl_ticks, pl_bar_max, pl_color, pl_size)
    pyvis_graph(nodes, edges, title=title, pv_bar_max=pv_bar_max, pv_color=pv_color, pv_size_min=pv_size_min, pv_size_max=pv_size_max)

In [327]:
def display_major(major, bar_max=None):
    display_graph(nodes=major_nodes(major),
                  edges=major_prereqs(major), 
                  title=(major + " Course Tree"), 
                  pl_bar_max=bar_max)

def available_classes(classes, d=None,bar_max=None):
    sc = select_classes(classes)
    
    display_graph(nodes=sc[0],
                  edges=sc[1],
                  title=("Available Classes"),
                  pv_bar_max=10,
                  pl_bar_max=10,
                  pl_color=sc[2],
                  pl_size=3)

In [341]:
classes = ["MATH 1501", "CS 1331", "HTS 2100", "APPH 1050"]
available_classes(classes, title="selected_classes")

In [ ]:
search("ECE 3710")

In [344]:
display_major("MATH", bar_max=15)

In [ ]:
# [e[1] for e in edges if e[0] == "ENGL 1102"]

In [ ]:
len(nodes)

In [342]:
display_graph(title="all")

In [340]:
display_graph(nodes=ugrad_nodes(nodes), edges=ugrad_prereqs(edges), title="ugrad")

In [223]:
g = Network()
g.add_nodes([1,2,3], value=[10, 100, 400],
             title=['I am node 1', 'node 2 here', 'and im node 3'],
             x=[21.4, 54.2, 11.2],
             y=[100.2, 23.54, 32.1],
             label=['NODE 1', 'NODE 2', 'NODE 3'],
             color=['#00ff1e', '#162347', '#dd4b39'])

g.show("test.html")

In [275]:
4**(1/2)

2.0